# Usage Example

In [1]:
from birdy import WPSClient
from wps_tools.testing import get_target_url
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
# url = get_target_url("chickadee")
# print(f"Using chickadee on {url}")

In [3]:
chickadee = WPSClient("http://localhost:5004/wps")

SyntaxError: duplicate argument 'out_file' in function definition (base.py-1>, line 1)

In [ ]:
# NBVAL_IGNORE_OUTPUT
chickadee?